In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from keras.backend import clear_session

In [3]:
#The path of goal image. Please use your own goal image :)
images=["Lenna.png"]
#images=["image1.png","image2.png","image3.png"]

In [5]:
#dataset CIFAR10
#Please apply your own dataset :)
#train
train=mnist.load_data()[0][0]
train_label=mnist.load_data()[0][1]
train_label=train_label.reshape(len(train_label))
#test
x_test=mnist.load_data()[1][0]
y_test=mnist.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
data3=np.zeros([len(train),28,28])
for n in range(len(train)):
    data3[n]=Image.fromarray(train[n].astype("uint8"))
train=data3

In [8]:
#image size
M=train.shape[1]
N=train.shape[2]
#C=train.shape[3]
C=1

In [ ]:
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[6]

for goal in images:
    #goal image I, equation(3)
    I=Image.open(goal).resize([M,N]).convert("L")
    
    AUC_list=[]
    BA_list=[]
    Error_list=[]
    # Q is corresponding to Fig.6.
    for Q in convo:
        AUC=[]
        BA=[]
        Error=[]
        # e is random_state. We use 5 different random seed.
        for e in range(1):
            ensemble=[]
            #Rotation
            for rot in degree:
                error_record=[]
                
                
                balanced_accuracy=np.zeros([10,9])
                AUC_score=np.zeros(10)
                for i in range(10):
                    clear_session()
                    x_train=train[train_label==i]
                    I_train=np.zeros([len(x_train),M,N])
                    for n in range(len(x_train)):
                        I_train[n]=I.rotate(rot)
                    I_train=I_train
                    
                    x_test=x_test.reshape(len(x_test),M,N,C)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  M, N,C)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        I_train = I_train.reshape(I_train.shape[0],
                                                  M, N,C)
                        input_shape = (M, N,C)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  C, M, N)
                        x_test = x_test.reshape(x_test.shape[0],C, M, N)
                        input_shape = (C, M, N)

                    x_train = x_train.astype('float32')
                    I_train = I_train.astype('float32')
                    x_train, x_valid, I_train, I_valid = train_test_split(x_train, I_train, test_size=0.3, random_state=e)
                    x_valid, x_threshold, I_valid, I_threshold = train_test_split(x_valid, I_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    
                    # model f:X → I
                    model = models.Sequential()
                    # Repeat convolution layer Q times
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(Q):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(C, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',
                                  loss='mean_absolute_error')
                    fit_callbacks = [
                        callbacks.EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                mode='min')
                    ]
                    # Train model
                    model.fit(x_train, I_train,
                              epochs=100,
                              batch_size=16,
                              shuffle=True,
                              validation_data=(x_valid, I_valid),callbacks=fit_callbacks)
                    #f_threshold = ITN output from x_threshold
                    f_threshold=model.predict(x_threshold)
                    f_test=model.predict(x_test)
                    I_train=I_train.reshape(I_train.shape[0],int(M*N) ,C)
                    goal_image=I_train[0]
                    f_threshold=f_threshold.reshape(f_threshold.shape[0],int(M*N),C)
                    f_test=f_test.reshape(f_test.shape[0],int(M*N),C)
                    # equation (12)
                    Eth=np.zeros(len(f_threshold))
                    
                    for n in range(len(f_threshold)):
                        Eth[n]=mean_absolute_error(goal_image,f_threshold[n])
                    Etest=np.zeros(len(f_test))
                    for n in range(len(f_test)):
                        Etest[n]=mean_absolute_error(goal_image,f_test[n])
                    error_record.append(Etest)
                    #y_one=np.ones(len(Eth))
                    #y_outlier=np.zeros(len(Etest))
                    #data=np.concatenate([Eth,Etest])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([Eth,Etest],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    #multiply construction error by (-1)
                    AUC_score[i]=roc_auc_score(y_label,(-1)*Etest)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(Eth)*p)+1
                        #print(Eth[Eth.argsort()[-threshold]])
                        y_pred[Etest<=Eth[Eth.argsort()[-threshold]]]=1
                        #Etest[Etest>Eth[Eth.argsort()[-threshold]]
                        balanced_accuracy[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                ensemble.append(error_record)
            AUC.append(AUC_score)
            BA.append(balanced_accuracy)
            Error.append(ensemble)
        AUC_list.append(AUC)
        BA_list.append(BA)
        Error_list.append(Error)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

In [12]:
np.array(BA_list).mean(axis=0)

array([[[0.54778271, 0.96649735, 0.9504231 , 0.93286461, 0.91476311,
         0.88595864, 0.85505792, 0.83436128, 0.80692113],
        [0.858397  , 0.98142599, 0.96918735, 0.94778133, 0.91722633,
         0.88567748, 0.86420438, 0.82202643, 0.79911894],
        [0.64938169, 0.91584938, 0.93046767, 0.92717777, 0.90331428,
         0.89134796, 0.86596572, 0.85090788, 0.82884407],
        [0.50395434, 0.82387416, 0.86633058, 0.879437  , 0.87499587,
         0.86769127, 0.84776594, 0.83770857, 0.82191103],
        [0.65995786, 0.91961641, 0.91409973, 0.88607442, 0.87565828,
         0.85688388, 0.84373062, 0.8239231 , 0.80393908],
        [0.60137124, 0.88913112, 0.91260898, 0.90627222, 0.89877991,
         0.87942965, 0.86224647, 0.8290504 , 0.80507724],
        [0.91604777, 0.9710876 , 0.94721709, 0.91857356, 0.89193137,
         0.87190825, 0.84743327, 0.83136433, 0.79906054],
        [0.57545698, 0.88013617, 0.88581965, 0.88207974, 0.87447437,
         0.85864627, 0.84517418, 0.8202976

In [13]:
table=[]
for i in range(10):
    raw=[]
    y_label=np.zeros(len(y_test))
    y_label[y_test==i]=1
    raw.append(roc_auc_score(y_label,(-1)*ensemble[0][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[1][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[2][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[3][i]))
    raw.append(roc_auc_score(y_label,(-1)*np.array(ensemble).sum(axis=0)[i]))
    table.append(raw)

In [14]:
pd.DataFrame(table)

,0,1,2,3,4
0,0.995272,0.990488,0.994858,0.994561,0.995727
1,0.997395,0.997077,0.997161,0.997430,0.997864
2,0.972840,0.976773,0.960781,0.981730,0.980738
3,0.947213,0.945728,0.950821,0.949653,0.960960
4,0.955893,0.965871,0.949617,0.965664,0.962905
5,0.981673,0.976440,0.973869,0.970290,0.981680
6,0.995628,0.994153,0.993835,0.996261,0.996669
7,0.953012,0.968814,0.961982,0.960753,0.967380
8,0.974515,0.961230,0.970091,0.975329,0.977662
9,0.972898,0.983266,0.976388,0.978468,0.981858
